In [2]:
# ==========================================================
# High-Fidelity Artwork Analyzer v12
# ==========================================================
# Local, fully dynamic, and saves:
#   - overlay_final_composite.png
#   - curatorial_summary.txt
# ==========================================================

import os, cv2, numpy as np, time, textwrap
from PIL import Image
from math import atan2, degrees

# ----------------------------------------------------------
# Utility
# ----------------------------------------------------------
def load_rgb(path):
    img = Image.open(path).convert("RGB")
    return np.array(img)

def ensure_dir(p): os.makedirs(p, exist_ok=True)
def to_uint8(img): return np.clip(img*255.0,0,255).astype(np.uint8)

# ----------------------------------------------------------
# Color Analysis
# ----------------------------------------------------------
def analyze_color_opencv(img, k=6):
    H,W=img.shape[:2]
    lab=cv2.cvtColor(img,cv2.COLOR_RGB2LAB)
    Z=lab.reshape(-1,3).astype(np.float32)
    crit=(cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER,50,0.3)
    _,labels,centers=cv2.kmeans(Z,k,None,crit,10,cv2.KMEANS_PP_CENTERS)
    lbl2=labels.reshape(H,W)
    rgb_centers=cv2.cvtColor(centers.reshape(-1,1,3).astype(np.uint8),cv2.COLOR_Lab2RGB).reshape(-1,3)
    masks=[(lbl2==i).astype(np.uint8)*255 for i in range(k)]
    a,b=centers[:,1]-128,centers[:,2]-128
    hues=(np.degrees(np.arctan2(b,a))+360)%360
    warm=((hues<90)|(hues>330)).sum()/len(hues)
    cool=((hues>180)&(hues<300)).sum()/len(hues)
    harmony="mixed"
    if np.ptp(hues)<35: harmony="analogous"
    elif any(abs(hues[i]-hues[j])%360>150 for i in range(len(hues)) for j in range(i+1,len(hues))):
        harmony="complementary"
    mean_deltaE=float(np.mean(np.abs(np.diff(centers,axis=0))))
    palette_entropy=float(-np.sum(np.histogram(labels,bins=k)[0]/(H*W)*np.log2(np.histogram(labels,bins=k)[0]/(H*W)+1e-6)))
    return dict(
        palette_rgb=[tuple(map(int,c)) for c in rgb_centers],
        masks=masks,
        color_variability=float(np.std(centers,0).mean()),
        warm_ratio=float(warm),
        cool_ratio=float(cool),
        harmony=harmony,
        mean_deltaE=mean_deltaE,
        palette_entropy=palette_entropy
    )

# ----------------------------------------------------------
# Texture, Saliency, Composition, Illumination
# ----------------------------------------------------------
def analyze_texture(img):
    g=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    lap=cv2.Laplacian(g,cv2.CV_64F)
    val=float(lap.var())
    roughness=float(np.std(lap))
    gabor_kernels=[cv2.getGaborKernel((15,15),3,np.pi/4*i,10,0.5,0,ktype=cv2.CV_32F) for i in range(4)]
    responses=[cv2.filter2D(g,cv2.CV_32F,k)**2 for k in gabor_kernels]
    anisotropy=float(np.std([r.mean() for r in responses])/np.mean([r.mean() for r in responses]+[1e-6]))
    return dict(laplacian_var=val,gabor_anisotropy=anisotropy,lbp_roughness=roughness,lbp_entropy=np.log(val+1))

def spectral_saliency(grayf):
    fft=np.fft.fft2(grayf)
    loga=np.log(np.abs(fft)+1e-8)
    spec=np.exp((loga-cv2.blur(loga,(3,3)))+1j*np.angle(fft))
    sal=np.abs(np.fft.ifft2(spec))**2
    sal=cv2.GaussianBlur(sal,(9,9),2)
    sal=(sal-sal.min())/(sal.max()-sal.min()+1e-6)
    return (sal*255).astype(np.uint8)

def subject_from_saliency(sal):
    H,W=sal.shape
    _,th=cv2.threshold(sal,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    cnts,_=cv2.findContours(th,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    if cnts:
        x,y,w,h=cv2.boundingRect(max(cnts,key=cv2.contourArea))
    else: x,y,w,h=W//4,H//4,W//2,H//2
    mask=np.zeros((H,W),np.uint8); cv2.rectangle(mask,(x,y),(x+w,y+h),255,-1)
    return dict(mask=mask,bbox=(x,y,w,h))

def analyze_saliency(img):
    g=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY).astype(np.float32)
    sal=spectral_saliency(g)
    sub=subject_from_saliency(sal)
    Y,X=np.indices(sal.shape); w=sal.astype(np.float32)+1
    cx,cy=int((X*w).sum()/w.sum()),int((Y*w).sum()/w.sum())
    return dict(saliency_map=sal,focal=(cx,cy),
                subject_mask=sub["mask"],subject_bbox=sub["bbox"])

def analyze_composition(img):
    g=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    H,W=g.shape; edges=cv2.Canny(g,100,200)
    linesP=cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength=min(H,W)//10,maxLineGap=15)
    lines=[tuple(map(int,l[0])) for l in linesP] if linesP is not None else []
    lw,rw=float(np.sum(edges[:,:W//2])),float(np.sum(edges[:,W//2:]))
    bal=(rw-lw)/(lw+rw+1e-6)
    thirds_delta=abs(((W//3)-(W/2))/W)
    golden_delta=abs((0.382*W - W/2)/W)
    sym=float(np.mean(np.abs(g[:,::-1]-g)))
    return dict(lines=lines,edge_balance=bal,thirds_delta=thirds_delta,
                golden_delta=golden_delta,symmetry=sym,vanishing_point=(W//2,H//2))

def illumination(img):
    L=cv2.cvtColor(img,cv2.COLOR_RGB2LAB)[:,:,0].astype(np.float32)
    mu,sig=L.mean(),L.std()+1e-6
    skew=float((((L-mu)/sig)**3).mean())
    gx,gy=cv2.Sobel(L,cv2.CV_32F,1,0),cv2.Sobel(L,cv2.CV_32F,0,1)
    hi=(L>np.percentile(L,90)).astype(np.uint8)
    if hi.sum()>0:
        gx,gy=(gx*hi).sum(),(gy*hi).sum()
        ang=float((degrees(atan2(-gy,-gx))+360)%360)
    else: ang=None
    return dict(lightness_skew=skew,light_direction_deg=ang)

# ----------------------------------------------------------
# Curatorial Text Generator
# ----------------------------------------------------------
def generate_curatorial_summary(metrics):
    C, T, P, I = (metrics["color"], metrics["texture"],
                  metrics["composition"], metrics["illumination"])

    if C["harmony"] == "analogous":
        color_tone = "a restrained palette of subtle continuity"
    elif C["harmony"] == "complementary":
        color_tone = "a lively chromatic counterpoint"
    else:
        color_tone = "a flexible palette balancing contrast and unity"

    if C["mean_deltaE"] > 25:
        contrast_phrase = "marked by strong contrasts"
    elif C["mean_deltaE"] > 10:
        contrast_phrase = "balanced between tonal unity and variety"
    else:
        contrast_phrase = "gentle in its chromatic gradations"

    if T["gabor_anisotropy"] > 0.5:
        texture_line = "The surface carries directional energy and gesture."
    else:
        texture_line = "The surface remains smooth and tonally cohesive."

    if abs(P["edge_balance"]) < 0.1:
        balance_phrase = "Spatial balance remains grounded and stable."
    else:
        balance_phrase = "The composition carries a deliberate asymmetry."

    if I["light_direction_deg"] is not None:
        illum_phrase = f"Light enters around {int(I['light_direction_deg'])}°, guiding focus."
    else:
        illum_phrase = "Light diffuses evenly, maintaining tonal calm."

    mood = "Luminous" if I["lightness_skew"] > 0.3 else "Somber" if I["lightness_skew"] < -0.3 else "Balanced"

    micro_summary = textwrap.fill(
        f"Color: {C['harmony']} harmony (ΔE≈{C['mean_deltaE']:.1f}), "
        f"Texture variance {T['laplacian_var']:.1f}, "
        f"Anisotropy {T['gabor_anisotropy']:.2f}, "
        f"Composition balance {P['edge_balance']:.2f}, "
        f"Light direction {I['light_direction_deg']}.",
        width=85
    )

    curatorial = textwrap.fill(
        f"This work reveals {color_tone}, {contrast_phrase}. "
        f"{texture_line} {balance_phrase} {illum_phrase} "
        f"The overall mood is {mood.lower()}, "
        f"suggesting a deliberate interplay of control and emotion. "
        f"The artist invites a slow, attentive gaze — "
        f"a chance to notice how color, gesture, and structure breathe together.",
        width=85
    )

    return micro_summary, curatorial

# ----------------------------------------------------------
# Visualization
# ----------------------------------------------------------
def make_composite(img, metrics):
    base = img.astype(np.float32) / 255.0
    H, W = base.shape[:2]

    # --- Laplacian texture intensity ---
    g = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    lap = np.abs(cv2.Laplacian(g, cv2.CV_32F))
    lap = cv2.normalize(lap, None, 0, 1, cv2.NORM_MINMAX)

    # --- Saliency map normalization ---
    sal = metrics["saliency"]["saliency_map"].astype(np.float32)
    sal = cv2.resize(sal, (W, H))
    sal = cv2.normalize(sal, None, 0, 1, cv2.NORM_MINMAX)

    # --- Create matched 3-channel layers ---
    zeros = np.zeros((H, W), np.float32)
    tex_layer = np.dstack([lap * 0.7, lap * 0.3, zeros]) * 0.3
    sal_layer = np.dstack([sal * 0.4, sal * 0.1, sal * 0.1]) * 0.3

    # --- Blend base with overlays ---
    comp = base * 0.9 + tex_layer + sal_layer
    comp = np.clip(comp, 0, 1)

    # --- Draw subject contour and focal point ---
    mask = metrics["saliency"]["subject_mask"]
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    comp_bgr = (comp[:, :, ::-1] * 255).astype(np.uint8).copy()

    if cnts:
        cv2.drawContours(comp_bgr, cnts, -1, (255, 0, 255), 2)
    fx, fy = metrics["saliency"]["focal"]
    cv2.circle(comp_bgr, (int(fx), int(fy)), 6, (0, 0, 255), -1)

    return comp_bgr


# ----------------------------------------------------------
# Pipeline
# ----------------------------------------------------------
def analyze_artwork_high_fidelity(path):
    start=time.time()
    img=load_rgb(path)
    color=analyze_color_opencv(img)
    texture=analyze_texture(img)
    sal=analyze_saliency(img)
    comp=analyze_composition(img)
    illum=illumination(img)
    metrics=dict(color=color,texture=texture,saliency=sal,composition=comp,illumination=illum)

    out_dir=os.path.splitext(path)[0]+"_analysis"
    ensure_dir(out_dir)

    # composite
    compo=make_composite(img,metrics)
    out_png=os.path.join(out_dir,"overlay_final_composite.png")
    cv2.imwrite(out_png,compo)

    # text
    micro,curatorial=generate_curatorial_summary(metrics)
    out_txt=os.path.join(out_dir,"curatorial_summary.txt")
    with open(out_txt,"w") as f:
        f.write("--- MICRO SUMMARY ---\n")
        f.write(micro+"\n\n--- CURATORIAL SUMMARY ---\n")
        f.write(curatorial)

    print(f"\n✅ Analysis complete.\nSaved to: {out_dir}\n⏱️ Runtime: {time.time()-start:.2f}s")

# ----------------------------------------------------------
# Run Example
# ----------------------------------------------------------
if __name__=="__main__":
    IMAGE_PATH="/Users/alievanayasso/Documents/SlowMA/Modigliani.jpg"
    analyze_artwork_high_fidelity(IMAGE_PATH)



✅ Analysis complete.
Saved to: /Users/alievanayasso/Documents/SlowMA/Modigliani_analysis
⏱️ Runtime: 11.61s
